In [1]:
#learn kung fu
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import mason_functions as mf

import os
import evaluate
import warnings
warnings.filterwarnings('ignore')

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [2]:
#define my sql query to access the relational database
sql = '''
SELECT * 
FROM predictions_2017
LEFT JOIN properties_2017 USING(parcelid)
WHERE propertylandusetypeid = 261 or propertylandusetypeid = 279
'''

#define my url
url = mf.get_db_url('zillow')

#read the information from a database into a dataframe
#I also will not continuously query the database; instead, I will write the data to a .csv (in case the kernel gotta go)
if os.path.isfile('zillow_actual.csv'):
    df = pd.read_csv('zillow_actual.csv', index_col = 0)
else:
    df = pd.read_sql(sql, url)
    df.to_csv('zillow_actual.csv')

In [3]:
#at a glance
pd.options.display.max_columns = 69
pd.options.display.max_rows = 69
print(df.info)

<bound method DataFrame.info of        parcelid     id  logerror transactiondate     id.1  \
0      14297519      0  0.025595      2017-01-01  1727539   
1      17052889      1  0.055619      2017-01-01  1387261   
2      14186244      2  0.005383      2017-01-01    11677   
3      12177905      3 -0.103410      2017-01-01  2288172   
4      12095076      6 -0.001011      2017-01-01   781532   
...         ...    ...       ...             ...      ...   
52437  11000655  77609  0.020615      2017-09-20   673515   
52438  17239384  77610  0.013209      2017-09-21  2968375   
52439  12773139  77611  0.037129      2017-09-21  1843709   
52440  12826780  77612  0.007204      2017-09-25  1187175   
52441  13083743  77613 -0.197755      2018-05-25  2455685   

       airconditioningtypeid  architecturalstyletypeid  basementsqft  \
0                        NaN                       NaN           NaN   
1                        NaN                       NaN           NaN   
2                  

In [4]:
#double-take
df.head()

,parcelid,id,logerror,transactiondate,id.1,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,finishedfloor1squarefeet,calculatedfinishedsquarefeet,finishedsquarefeet12,finishedsquarefeet13,finishedsquarefeet15,finishedsquarefeet50,finishedsquarefeet6,fips,fireplacecnt,fullbathcnt,garagecarcnt,garagetotalsqft,hashottuborspa,heatingorsystemtypeid,latitude,longitude,lotsizesquarefeet,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,propertycountylandusecode,propertylandusetypeid,propertyzoningdesc,rawcensustractandblock,regionidcity,regionidcounty,regionidneighborhood,regionidzip,roomcnt,storytypeid,threequarterbathnbr,typeconstructiontypeid,unitcnt,yardbuildingsqft17,yardbuildingsqft26,yearbuilt,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,14297519,0,0.025595,2017-01-01,1727539,NaN,NaN,NaN,3.5,4.0,NaN,NaN,3.5,NaN,NaN,3100.0,3100.0,NaN,NaN,NaN,NaN,6059.0,NaN,3.0,2.0,633.0,NaN,NaN,33634931.0,-117869207.0,4506.0,NaN,NaN,NaN,NaN,NaN,122,261.0,NaN,6.059063e+07,53571.0,1286.0,NaN,96978.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1998.0,NaN,NaN,485713.0,1023282.0,2016.0,537569.0,11013.72,NaN,NaN,6.059063e+13
1,17052889,1,0.055619,2017-01-01,1387261,NaN,NaN,NaN,1.0,2.0,NaN,NaN,1.0,NaN,1465.0,1465.0,1465.0,NaN,NaN,1465.0,NaN,6111.0,1.0,1.0,1.0,0.0,NaN,NaN,34449266.0,-119281531.0,12647.0,NaN,NaN,NaN,NaN,NaN,1110,261.0,NaN,6.111001e+07,13091.0,2061.0,NaN,97099.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,1967.0,1.0,NaN,88000.0,464000.0,2016.0,376000.0,5672.48,NaN,NaN,6.111001e+13
2,14186244,2,0.005383,2017-01-01,11677,NaN,NaN,NaN,2.0,3.0,NaN,NaN,2.0,NaN,NaN,1243.0,1243.0,NaN,NaN,NaN,NaN,6059.0,NaN,2.0,2.0,440.0,NaN,NaN,33886168.0,-117823170.0,8432.0,1.0,NaN,NaN,NaN,1.0,122,261.0,NaN,6.059022e+07,21412.0,1286.0,NaN,97078.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,1962.0,1.0,NaN,85289.0,564778.0,2016.0,479489.0,6488.30,NaN,NaN,6.059022e+13
3,12177905,3,-0.103410,2017-01-01,2288172,NaN,NaN,NaN,3.0,4.0,NaN,8.0,3.0,NaN,NaN,2376.0,2376.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34245180.0,-118240722.0,13038.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,LCR110000*,6.037300e+07,396551.0,3101.0,NaN,96330.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1970.0,NaN,NaN,108918.0,145143.0,2016.0,36225.0,1777.51,NaN,NaN,6.037300e+13
4,12095076,6,-0.001011,2017-01-01,781532,1.0,NaN,NaN,3.0,4.0,NaN,9.0,3.0,NaN,NaN,2962.0,2962.0,NaN,NaN,NaN,NaN,6037.0,NaN,3.0,NaN,NaN,NaN,2.0,34145202.0,-118179824.0,63000.0,1.0,NaN,NaN,NaN,1.0,0101,261.0,PSR2,6.037461e+07,47019.0,3101.0,274684.0,96293.0,0.0,NaN,NaN,NaN,1.0,NaN,NaN,1950.0,NaN,NaN,276684.0,773303.0,2016.0,496619.0,9516.26,NaN,NaN,6.037461e+13


In [5]:
#summary statistics
df.describe().T

,count,mean,std,min,25%,50%,75%,max
parcelid,52442.0,1.299912e+07,3.411412e+06,1.071186e+07,1.151030e+07,1.257766e+07,1.412951e+07,1.676878e+08
id,52442.0,3.886063e+04,2.234844e+04,0.000000e+00,1.954625e+04,3.892750e+04,5.809350e+04,7.761300e+04
logerror,52442.0,1.829437e-02,1.775018e-01,-4.655420e+00,-2.463935e-02,7.009984e-03,4.069098e-02,5.262999e+00
id.1,52442.0,1.497288e+06,8.594413e+05,3.490000e+02,7.579560e+05,1.500836e+06,2.241564e+06,2.982270e+06
airconditioningtypeid,13638.0,2.438041e+00,3.846176e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.300000e+01
architecturalstyletypeid,70.0,7.100000e+00,2.665670e+00,2.000000e+00,7.000000e+00,7.000000e+00,7.000000e+00,2.100000e+01
basementsqft,47.0,6.789787e+02,7.118252e+02,3.800000e+01,2.635000e+02,5.120000e+02,8.095000e+02,3.560000e+03
bathroomcnt,52442.0,2.299397e+00,1.022764e+00,0.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,1.800000e+01
bedroomcnt,52442.0,3.300675e+00,9.490858e-01,0.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,1.400000e+01
buildingclasstypeid,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#Nulls?
df.isnull().any()

parcelid                        False
id                              False
logerror                        False
transactiondate                 False
id.1                            False
airconditioningtypeid            True
architecturalstyletypeid         True
basementsqft                     True
bathroomcnt                     False
bedroomcnt                      False
buildingclasstypeid              True
buildingqualitytypeid            True
calculatedbathnbr                True
decktypeid                       True
finishedfloor1squarefeet         True
calculatedfinishedsquarefeet     True
finishedsquarefeet12             True
finishedsquarefeet13             True
finishedsquarefeet15             True
finishedsquarefeet50             True
finishedsquarefeet6              True
fips                            False
fireplacecnt                     True
fullbathcnt                      True
garagecarcnt                     True
garagetotalsqft                  True
hashottubors

In [7]:
#how many nulls and where?
df.isnull().sum()

parcelid                            0
id                                  0
logerror                            0
transactiondate                     0
id.1                                0
airconditioningtypeid           38804
architecturalstyletypeid        52372
basementsqft                    52395
bathroomcnt                         0
bedroomcnt                          0
buildingclasstypeid             52442
buildingqualitytypeid           18701
calculatedbathnbr                 137
decktypeid                      52053
finishedfloor1squarefeet        48061
calculatedfinishedsquarefeet       82
finishedsquarefeet12              247
finishedsquarefeet13            52442
finishedsquarefeet15            52442
finishedsquarefeet50            48061
finishedsquarefeet6             52277
fips                                0
fireplacecnt                    45199
fullbathcnt                       137
garagecarcnt                    34427
garagetotalsqft                 34427
hashottubors

In [8]:
#quick reminder on total number observations
mf.add_commas(len(df))

'52,442'

In [9]:
#what percentage of airconditioningtypeid is null? 
#74 percent
df.airconditioningtypeid.isnull().sum() / len(df)

0.7399412684489531

In [10]:
#let's drop add this to list of columns to drop-- and everything else with over 70% null values
null_and_void = []

In [11]:
#index all of my columns into a list
df_columns = df.columns.to_list()

In [12]:
#here's the list
df_columns

['parcelid',
 'id',
 'logerror',
 'transactiondate',
 'id.1',
 'airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'bathroomcnt',
 'bedroomcnt',
 'buildingclasstypeid',
 'buildingqualitytypeid',
 'calculatedbathnbr',
 'decktypeid',
 'finishedfloor1squarefeet',
 'calculatedfinishedsquarefeet',
 'finishedsquarefeet12',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fips',
 'fireplacecnt',
 'fullbathcnt',
 'garagecarcnt',
 'garagetotalsqft',
 'hashottuborspa',
 'heatingorsystemtypeid',
 'latitude',
 'longitude',
 'lotsizesquarefeet',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'propertycountylandusecode',
 'propertylandusetypeid',
 'propertyzoningdesc',
 'rawcensustractandblock',
 'regionidcity',
 'regionidcounty',
 'regionidneighborhood',
 'regionidzip',
 'roomcnt',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'unitcnt',
 'yardbuildingsqft17',
 'yardbuildings

In [13]:
#Maybe I don't want to have columns where over 70% of the dataframe is missing
for column in df_columns:
    if df[f'{column}'].isnull().sum() / len(df) >= .7:
        null_and_void.append(column)

In [14]:
#what's here?
null_and_void

['airconditioningtypeid',
 'architecturalstyletypeid',
 'basementsqft',
 'buildingclasstypeid',
 'decktypeid',
 'finishedfloor1squarefeet',
 'finishedsquarefeet13',
 'finishedsquarefeet15',
 'finishedsquarefeet50',
 'finishedsquarefeet6',
 'fireplacecnt',
 'hashottuborspa',
 'poolcnt',
 'poolsizesum',
 'pooltypeid10',
 'pooltypeid2',
 'pooltypeid7',
 'storytypeid',
 'threequarterbathnbr',
 'typeconstructiontypeid',
 'yardbuildingsqft17',
 'yardbuildingsqft26',
 'numberofstories',
 'fireplaceflag',
 'taxdelinquencyflag',
 'taxdelinquencyyear']

Maybe I don't want to have columns where over half of the dataframe is missing. Or, maybe I don't want rows where my data is missing. If I drop the columns, I will lose what are seemingly important columns, e.g., 'typeconstructiontypeid', 'poolcnt', or 'storytypeid.' If I drop the rows, I will have that many less observations to work with. I will try one approach and evaluate my model's performance. When I have time, I will come back and approach the other way or in some compromising manner.

In [15]:
#drop columns
df = df.drop(columns = null_and_void)

In [16]:
#quick look at what's left
df.columns

Index(['parcelid', 'id', 'logerror', 'transactiondate', 'id.1', 'bathroomcnt',
       'bedroomcnt', 'buildingqualitytypeid', 'calculatedbathnbr',
       'calculatedfinishedsquarefeet', 'finishedsquarefeet12', 'fips',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
       'heatingorsystemtypeid', 'latitude', 'longitude', 'lotsizesquarefeet',
       'propertycountylandusecode', 'propertylandusetypeid',
       'propertyzoningdesc', 'rawcensustractandblock', 'regionidcity',
       'regionidcounty', 'regionidneighborhood', 'regionidzip', 'roomcnt',
       'unitcnt', 'yearbuilt', 'structuretaxvaluedollarcnt',
       'taxvaluedollarcnt', 'assessmentyear', 'landtaxvaluedollarcnt',
       'taxamount', 'censustractandblock'],
      dtype='object')

In [17]:
#another look at remaining nulls
df.isnull().sum()

parcelid                            0
id                                  0
logerror                            0
transactiondate                     0
id.1                                0
bathroomcnt                         0
bedroomcnt                          0
buildingqualitytypeid           18701
calculatedbathnbr                 137
calculatedfinishedsquarefeet       82
finishedsquarefeet12              247
fips                                0
fullbathcnt                       137
garagecarcnt                    34427
garagetotalsqft                 34427
heatingorsystemtypeid           18506
latitude                            0
longitude                           0
lotsizesquarefeet                 369
propertycountylandusecode           0
propertylandusetypeid               0
propertyzoningdesc              18593
rawcensustractandblock              0
regionidcity                     1037
regionidcounty                      0
regionidneighborhood            33409
regionidzip 

In [21]:
#let's start here
df.columns[df.isnull().sum() > 100]

Index(['buildingqualitytypeid', 'calculatedbathnbr', 'finishedsquarefeet12',
       'fullbathcnt', 'garagecarcnt', 'garagetotalsqft',
       'heatingorsystemtypeid', 'lotsizesquarefeet', 'propertyzoningdesc',
       'regionidcity', 'regionidneighborhood', 'unitcnt', 'yearbuilt',
       'censustractandblock'],
      dtype='object')

In [25]:
#assign variable to focus data
lotta_nulls = df.columns[df.isnull().sum() > 100]

In [30]:
#a closer look
df[lotta_nulls].isnull().sum()

buildingqualitytypeid    18701
calculatedbathnbr          137
finishedsquarefeet12       247
fullbathcnt                137
garagecarcnt             34427
garagetotalsqft          34427
heatingorsystemtypeid    18506
lotsizesquarefeet          369
propertyzoningdesc       18593
regionidcity              1037
regionidneighborhood     33409
unitcnt                  18594
yearbuilt                  116
censustractandblock        123
dtype: int64

In [45]:
#under the glass
for column in lotta_nulls:
    print(f'{column}')
    print('---')
    print('Number of unique values')
    print('-----------------------')
    print(df[f'{column}'].nunique())
    print('---')
    print('Value Counts')
    print('------------')
    print(df[f'{column}'].value_counts(dropna = False).sort_index())
    print('=============')

buildingqualitytypeid
---
Number of unique values
-----------------------
12
---
Value Counts
------------
1.0         8
2.0         5
3.0       152
4.0      8157
5.0      1513
6.0     10289
7.0      3427
8.0      8244
9.0      1121
10.0      230
11.0      514
12.0       81
NaN     18701
Name: buildingqualitytypeid, dtype: int64
calculatedbathnbr
---
Number of unique values
-----------------------
21
---
Value Counts
------------
1.0      9568
1.5       834
2.0     21894
2.5      3926
3.0     10673
3.5       918
4.0      2227
4.5       687
5.0       803
5.5       223
6.0       322
6.5        47
7.0        88
7.5        16
8.0        53
8.5         3
9.0        13
10.0        5
11.0        3
13.0        1
18.0        1
NaN       137
Name: calculatedbathnbr, dtype: int64
finishedsquarefeet12
---
Number of unique values
-----------------------
4716
---
Value Counts
------------
128.0        1
152.0        1
240.0        1
280.0        1
288.0        1
          ... 
17245.0      1
18654.0